# DATASCI 210: Predicting The Next Pitch

Lorem ipsum delorum.

In [10]:
import psycopg2

import numpy as np
import pandas as pd

from sklearn import cross_validation

from config import REDSHIFT_CONFIG
from src.features import *
from src.utils import *
from src.validation import *

In [20]:
conn = create_conn(config=REDSHIFT_CONFIG)
cur = conn.cursor()

In [23]:
colnames = [header[0] for header in cur.description]

In [24]:
pitch_df = pd.DataFrame(rows)
pitch_df.columns = colnames
pitch_df.head()

,game_id,num,pitcher,batter,b,s,des,id,type,x,...,ay,az,break_y,break_angle,break_length,pitch_type,type_confidence,spin_dir,spin_rate,zone
0,2006/06/04/anamlb-clemlb-1,1,282332,408210,0,0,Ball,3,B,83.260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006/06/04/anamlb-clemlb-1,1,282332,408210,1,1,Ball,5,B,62.660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006/06/04/anamlb-clemlb-1,1,282332,408210,2,2,Foul,7,S,100.430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006/06/04/anamlb-clemlb-1,1,282332,408210,3,3,"In play, run(s)",9,X,105.580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006/06/04/anamlb-clemlb-1,2,282332,111851,0,1,Swinging Strike,15,S,77.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
"""
cur.close()
conn.close()
"""

'\ncur.close()\nconn.close()\n'

In [25]:
def get_available_rs_tables(cursor, *args, **kwargs):
    query = """select distinct(tablename) from pg_table_def where schemaname = 'public';"""
    cur = cursor
    cur.execute(query)
    rows = cur.fetchall()
    header = [colnames[0] for colnames in cur.description]
    return header, rows

In [26]:
available_tables_header, available_tables_data = get_available_rs_tables(cur)

In [28]:
available_table_headers, available_table_data

(['tablename'],
 [('atbat',),
  ('atbat_pkey',),
  ('game',),
  ('game_pkey',),
  ('hitchart',),
  ('hitchart_pkey',),
  ('pitch',),
  ('pitch_pkey',),
  ('player',),
  ('player_pkey',)])

In [ ]:
# close_rs_conn(cur, conn)